In [1]:
import numpy as np
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt
import config
import json
import requests

In [3]:
airquality_key = config.a_key
covid_key = config.c_key
weather_key = config.w_key

In [20]:
subscriber_df = pd.read_csv('../Data/Subscriber_Info.csv')
subscriber_df

,First Name,Last Name,Address,City,State,Zipcode,Email
0,Katie,Hu,1234 Wallaby Ln,San Diego,CA,92123,khu@sandiego.edu
1,Trevor,Sauerbrey,3623 Ruette de Ville,San Diego,CA,921230,trevsauer@gmail.com
2,Grigor,Tashcyan,11426 Aldrich St,Whittier,CA,90606,grigortashchyan@gmail.com


In [3]:
city_id = ['San Diego', 'Whittier', 'Brooklyn']
city_id

['San Diego', 'Whittier', 'Brooklyn']

Air Quality API

In [22]:
url = "https://air-quality-by-api-ninjas.p.rapidapi.com/v1/airquality"

querystring = {"city":city_id}

headers = {
	"X-RapidAPI-Key": airquality_key,
	"X-RapidAPI-Host": "air-quality-by-api-ninjas.p.rapidapi.com"
}

a_response = requests.request("GET", url, headers=headers, params=querystring)

print(a_response.text)

{"CO": {"concentration": 297.07, "aqi": 3}, "NO2": {"concentration": 18.34, "aqi": 22}, "O3": {"concentration": 69.38, "aqi": 82}, "SO2": {"concentration": 6.38, "aqi": 9}, "PM2.5": {"concentration": 14.91, "aqi": 48}, "PM10": {"concentration": 16.87, "aqi": 15}, "overall_aqi": 82}


In [24]:
# Convert the response to a dictionary
a_data = a_response.json()

# Create a Pandas dataframe from the dictionary
a_df = pd.DataFrame(a_data)
a_df

,CO,NO2,O3,SO2,PM2.5,PM10,overall_aqi
concentration,297.07,18.34,69.38,6.38,14.91,16.87,82
aqi,3.00,22.00,82.00,9.00,48.00,15.00,82


Covid-19 API

In [18]:
# make the API call
# link to documentation: https://rapidapi.com/axisbits-axisbits-default/api/covid-19-statistics/
url = "https://covid-19-statistics.p.rapidapi.com/reports"

# optional parameter selected to querying the API
querystring = {"city_name":"San Diego"}

# API credentials
headers = {
	"X-RapidAPI-Key": covid_key,
	"X-RapidAPI-Host": "covid-19-statistics.p.rapidapi.com"
}

# get raw response query from the API
c_response = requests.request("GET", url, headers=headers, params=querystring)
print(c_response.text)

{"data":[{"date":"2023-02-12","confirmed":12031907,"deaths":100300,"recovered":0,"confirmed_diff":0,"deaths_diff":0,"recovered_diff":0,"last_update":"2023-02-13 04:20:44","active":11931607,"active_diff":0,"fatality_rate":0.0083,"region":{"iso":"USA","name":"US","province":"California","lat":"36.1162","long":"-119.6816","cities":[{"name":"San Diego","date":"2023-02-12","fips":6073,"lat":"33.03484597","long":"-116.7365326","confirmed":1059657,"deaths":5745,"confirmed_diff":0,"deaths_diff":0,"last_update":"2023-02-13 04:20:44"}]}}]}


In [19]:
# Convert the response to a dictionary
c_data = c_response.json()

# Create a Pandas dataframe from the dictionary
c_df = pd.DataFrame(c_data)

# Unpack the dictionaries in the column into separate columns
df_unpacked = pd.json_normalize(c_df['data'])

# merge the new columns back into the original dataframe
df_1 = c_df.merge(df_unpacked, left_index=True, right_index=True)

# drop the original column containing the dictionaries
df_2 = df_1.drop('data', axis=1)

# extract the dictionary from region.cities column for flattening
col_dic = df_2['region.cities'].iloc[0]

# turn the extracted dictionary into its on dataframe 
flatten_col = pd.DataFrame(col_dic)

# merge the two dataframes
result = df_2.merge(flatten_col, left_index=True, right_index=True)

# show the values of all the columns
row_values = result.iloc[0].to_dict()
row_values

{'date_x': '2023-02-12',
 'confirmed_x': 12031907,
 'deaths_x': 100300,
 'recovered': 0,
 'confirmed_diff_x': 0,
 'deaths_diff_x': 0,
 'recovered_diff': 0,
 'last_update_x': '2023-02-13 04:20:44',
 'active': 11931607,
 'active_diff': 0,
 'fatality_rate': 0.0083,
 'region.iso': 'USA',
 'region.name': 'US',
 'region.province': 'California',
 'region.lat': '36.1162',
 'region.long': '-119.6816',
 'region.cities': [{'name': 'San Diego',
   'date': '2023-02-12',
   'fips': 6073,
   'lat': '33.03484597',
   'long': '-116.7365326',
   'confirmed': 1059657,
   'deaths': 5745,
   'confirmed_diff': 0,
   'deaths_diff': 0,
   'last_update': '2023-02-13 04:20:44'}],
 'name': 'San Diego',
 'date_y': '2023-02-12',
 'fips': 6073,
 'lat': '33.03484597',
 'long': '-116.7365326',
 'confirmed_y': 1059657,
 'deaths_y': 5745,
 'confirmed_diff_y': 0,
 'deaths_diff_y': 0,
 'last_update_y': '2023-02-13 04:20:44'}

Weather API

In [20]:
url = "https://weather-by-api-ninjas.p.rapidapi.com/v1/weather"

headers = {
	"X-RapidAPI-Key": weather_key,
	"X-RapidAPI-Host": "weather-by-api-ninjas.p.rapidapi.com"
}
querystring = {"city":city_id}

w_response = requests.request("GET", url, headers=headers, params = querystring)

print(w_response.text)

{"cloud_pct": 0, "temp": 9, "feels_like": 5, "humidity": 34, "min_temp": 7, "max_temp": 10, "wind_speed": 6.69, "wind_degrees": 270, "sunrise": 1676289162, "sunset": 1676327252}


In [25]:
# Convert the response to a dictionary
w_data = w_response.json()

# Create a Pandas dataframe from the dictionary
w_df = pd.DataFrame(w_data, index = [0])
w_df

,cloud_pct,temp,feels_like,humidity,min_temp,max_temp,wind_speed,wind_degrees,sunrise,sunset
0,0,9,5,34,7,10,6.69,270,1676289162,1676327252
